In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0


    100% |████████████████████████████████| 332.1MB 50kB/s 
    100% |████████████████████████████████| 419kB 11.2MB/s 
    100% |████████████████████████████████| 61kB 29.7MB/s 
    100% |████████████████████████████████| 3.0MB 6.7MB/s 


In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, GRU, Dropout, Embedding
import numpy as np
import time


In [0]:
embedding_size = 8
start_note = 1
end_note = 2


In [0]:
note_to_label = {(0, 0, 0):0}
label_to_note = {0:(0, 0, 0)}

data = np.load('Data.npy')
tokenized = []
for song in data:
  s = [start_note]
  for row in song:
    note = tuple(row)
    if note == (0, 0, 0):
      continue
    if note not in note_to_label:
      note_to_label[note] = len(note_to_label) + 2
      label_to_note[note_to_label[note]] = note
    s.append(note_to_label[note])
  s.append(end_note)
  tokenized.append(s)

num_notes = len(note_to_label) + 3


In [0]:
x = keras.preprocessing.sequence.pad_sequences(tokenized, maxlen=maxlen, padding='post')
y = np.hstack([x[:, 1:], np.zeros((x.shape[0], 1), dtype=np.int)])
n = x.shape[0]
maxlen = x.shape[1]

indexes = np.random.permutation(n)
split_index = int(0.9 * n)
x_train = x[indexes[:split_index]]
x_val = x[indexes[split_index:]]
y_train = y[indexes[:split_index]]
y_val = y[indexes[split_index:]]


In [0]:
# model = keras.models.Sequential()
# model.add(Embedding(notes_size, embedding_size))
# model.add(GRU(16, return_sequences=True))
# model.add(Dropout(0.2))
# model.add(Dense(notes_size, activation='softmax'))
# model.summary()

class MyModel(keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.embedding = Embedding(num_notes, embedding_size)
    self.gru = GRU(16, return_sequences=True, return_state=True)
    self.dropout = Dropout(0.2)
    self.d = Dense(num_notes, activation='softmax')

  def call(self, x, state=None):
    x = self.embedding(x)
    x, output_state = self.gru(x, initial_state=state)
    x = self.dropout(x)
    x = self.d(x)
    return x, output_state

model = MyModel()


In [0]:
optimizer = keras.optimizers.Adam()
loss_metric_train = keras.metrics.Mean()
loss_metric_val = keras.metrics.Mean()

scc = keras.losses.SparseCategoricalCrossentropy()
def loss_function(y, output):
  return scc(y, output, sample_weight=tf.math.not_equal(y, 0))


In [0]:
@tf.function
def train_step(x, y, model):
  with tf.GradientTape() as tape:    
    output = model(x)[0]
    loss = loss_function(y, output)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  loss_metric_train(loss)


@tf.function
def val_step(x, y, model):
  output = model(x)[0]
  loss = loss_function(y, output)
  
  loss_metric_val(loss)


In [0]:
epochs = 200
batch_size = 1

# model.compile(optimizer='adam', loss=loss_function)
# model.fit(x, y, epochs=epochs, validation_split=0.1)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(x_val.shape[0])

start = time.time()
for epoch in range(epochs):
  for x, y in train_data:
    train_step(x, y, model)
  for x, y in val_data:
    val_step(x, y, model)
  if epoch % 1 == 0:
    end = time.time()
    print(f'Epoch {epoch}, Time: {end - start: .3f}s, Loss: {loss_metric_train.result(): .4f}, Validation Loss: {loss_metric_val.result(): .4f}')
    start = time.time()


Epoch 0, Time:  4.323s, Loss:  1.4027, Validation Loss:  1.0693
Epoch 1, Time:  2.216s, Loss:  1.2970, Validation Loss:  1.0636
Epoch 2, Time:  2.193s, Loss:  1.2594, Validation Loss:  1.0584
Epoch 3, Time:  2.187s, Loss:  1.2324, Validation Loss:  1.0438
Epoch 4, Time:  2.065s, Loss:  1.2043, Validation Loss:  1.0253
Epoch 5, Time:  2.013s, Loss:  1.1785, Validation Loss:  1.0080
Epoch 6, Time:  1.995s, Loss:  1.1564, Validation Loss:  0.9928
Epoch 7, Time:  1.992s, Loss:  1.1377, Validation Loss:  0.9798
Epoch 8, Time:  2.000s, Loss:  1.1218, Validation Loss:  0.9686
Epoch 9, Time:  1.998s, Loss:  1.1083, Validation Loss:  0.9591
Epoch 10, Time:  2.006s, Loss:  1.0966, Validation Loss:  0.9508
Epoch 11, Time:  1.998s, Loss:  1.0863, Validation Loss:  0.9434
Epoch 12, Time:  2.010s, Loss:  1.0772, Validation Loss:  0.9369
Epoch 13, Time:  2.014s, Loss:  1.0691, Validation Loss:  0.9310
Epoch 14, Time:  2.024s, Loss:  1.0616, Validation Loss:  0.9256
Epoch 15, Time:  2.033s, Loss:  1.0

In [0]:
def generate(model, k=3):
  result = [start_note]
  state = None
  while True:
    seq = np.array([[result[-1]]])
    output, state = model(seq, state=state)
    note = np.random.choice(np.argpartition(output[0][0], -k)[-k:])
    if note == 0:
      result.append(end_note)
    else:
      result.append(note)
    if result[-1] == end_note or len(result) >= maxlen:
      break
  return result[1:-1]


In [0]:
generated = generate(model)
print(len(generated))
print(generated)


179
[11, 16, 11, 16, 5, 9, 8, 5, 16, 3, 3, 6, 3, 14, 22, 7, 7, 14, 11, 3, 14, 22, 14, 22, 6, 7, 14, 6, 3, 14, 3, 6, 7, 7, 7, 14, 3, 14, 6, 14, 11, 14, 6, 3, 16, 11, 3, 11, 3, 11, 14, 6, 14, 3, 5, 16, 3, 6, 14, 3, 5, 10, 8, 5, 10, 5, 10, 8, 5, 10, 8, 9, 15, 9, 8, 10, 8, 9, 8, 5, 10, 5, 16, 3, 6, 3, 3, 11, 5, 11, 14, 6, 3, 7, 14, 6, 14, 3, 6, 14, 11, 14, 3, 6, 7, 6, 7, 7, 7, 14, 3, 14, 6, 14, 22, 7, 22, 6, 7, 6, 3, 14, 22, 7, 14, 11, 14, 6, 14, 22, 6, 7, 22, 14, 3, 6, 14, 11, 14, 3, 14, 22, 14, 22, 14, 6, 7, 6, 3, 16, 11, 3, 11, 14, 22, 14, 3, 5, 9, 8, 5, 10, 9, 15, 19, 9, 10, 8, 10, 5, 11, 14, 11, 3, 6, 3, 7, 6, 18]


In [0]:
generated_data = [generate(model) for i in range(10)]


In [0]:
new_data = keras.preprocessing.sequence.pad_sequences(generated_data, maxlen=maxlen, padding='post')
new_songs = np.array([[label_to_note[label] for label in row] for row in new_data])


In [0]:
np.save('new_songs', new_songs)
